In [1]:
import os
import shutil
import pandas as pd

In [2]:
df = pd.read_csv('trainTestFrame.csv')
df

,NIFTI_slice_name,sid_x,NIFTI_name,sid_y,split
0,sub-OAS30333_ses-d141_acq-PIB_pet__s6.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train
1,sub-OAS30333_ses-d141_acq-PIB_pet__s9.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train
2,sub-OAS30333_ses-d141_acq-PIB_pet__s22.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train
3,sub-OAS30333_ses-d141_acq-PIB_pet__s4.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train
4,sub-OAS30333_ses-d141_acq-PIB_pet__s7.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train
...,...,...,...,...,...
514345,sub-OAS30603_sess-d2253_CT__s1.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train
514346,sub-OAS30603_sess-d2253_CT__s3.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train
514347,sub-OAS30603_sess-d2253_CT__s4.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train
514348,sub-OAS30603_sess-d2253_CT__s5.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train


In [3]:
def renameDesc(desc):
    desc = desc.lower()
    if 'cwk' in desc or 'lwk' in desc or 'twk' in desc:
        return 'spinal'
    if 'localizer' in desc:
        return 'localizer'
    if 'phase' in desc:
        return 'phase'
    if 'field' in desc:
        return 'fieldmap'
    if 'rage' in desc or 'fspgr' in desc:
        return 'T1w'
    if 'cal' in desc:
        return 'calibration'
    if 'scout' in desc:
        return 'scout'
    if 'flair' in desc:
        return 'T2-FLAIR'
    if 'star' in desc:
        return 'T2star'
    if 'swi' in desc:
        return 'swi'
    if 'perf' in desc or 'pwi' in desc:
        return 'pwi'
    if 'asl' in desc:
        return 'asl'
    if 'dti' in desc:
        return 'dti'
    if 'cerebral_blood_flow' in desc or 'cbf' in desc:
        return 'cbf'
    if 't2' in desc:
        return 'T2w'
    if 'moco' in desc:
        return 'moco'
    if 'format' in desc:
        return 'mpr'
    if 'coronal' in desc or 't1' in desc:
        return 'T1w'
    if 'pet' in desc or 'ct' in desc:
        return 'NotMri'
    if 'bold' in desc:
        return 'bold'
    if 'dwi' in desc:
        return 'dwi'
    if 'gre' in desc:
        return 'gre'
    if 'angio' in desc:
        return 'angio'
    if 'min' in desc:
        return 'minip'
    return 'nan'

In [4]:
df['description']=df['NIFTI_name'].apply(renameDesc)

In [5]:
df['description'].unique()

array(['NotMri', 'T2star', 'fieldmap', 'T1w', 'bold', 'T2w', 'dwi',
       'T2-FLAIR', 'asl', 'swi', 'phase', 'gre', 'minip', 'angio'],
      dtype=object)

In [6]:
df[df['description']=='hippo']

,NIFTI_slice_name,sid_x,NIFTI_name,sid_y,split,description


In [7]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"PD":4,
             "SWI":5, "GRE":6, "T2*":7, "DWI":8, "ADC":9, "BOLD":10,
             "FlowSensitve":11,"PWI":12,"ASL":13,
             "DTI":14,"Other":15}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    dropLabels = ['moco','mpr','hippo','spinal','unknown']
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2','minIP','moco','other','survey', 'qflow','smartbrain'] #moco is a motion corrected series
    if description in dropLabels:
        return -1
    if description in otherCategories:
        return 15
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 4
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='pd':
        return 4
    if description =='swi' or description=='minip':
        return 5
    if description.lower() =='gre':
        return 6
    if description =='T2star':
        return 7
    if description =='dwi':
        return 8
    if description =='adc':
        return 9
    if description =='bold':
        return 10
    if description =='angio': #flowsensitive
        return 11
    if description =='pwi' or description=='cbf':
        return 12
    if 'asl' in description:
        return 13
    if description == 'dti':
        return 14
    return -1

In [8]:
df['label'] = df.apply(assignLabel,axis=1)

In [9]:
df['label'].value_counts()

label
15    123600
10     75425
2      62075
0      59750
8      50225
5      40650
7      35350
13     23500
3      18925
11     13925
6      10925
Name: count, dtype: int64

In [10]:
df[df['label']==-1]

,NIFTI_slice_name,sid_x,NIFTI_name,sid_y,split,description,label


In [11]:
df = df[df['label']!=-1]

In [12]:
train_df = df[df['split']=='train'].copy()
test_df = df[df['split']=='test'].copy()

In [13]:
train_df

,NIFTI_slice_name,sid_x,NIFTI_name,sid_y,split,description,label
0,sub-OAS30333_ses-d141_acq-PIB_pet__s6.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15
1,sub-OAS30333_ses-d141_acq-PIB_pet__s9.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15
2,sub-OAS30333_ses-d141_acq-PIB_pet__s22.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15
3,sub-OAS30333_ses-d141_acq-PIB_pet__s4.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15
4,sub-OAS30333_ses-d141_acq-PIB_pet__s7.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15
...,...,...,...,...,...,...,...
514345,sub-OAS30603_sess-d2253_CT__s1.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15
514346,sub-OAS30603_sess-d2253_CT__s3.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15
514347,sub-OAS30603_sess-d2253_CT__s4.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15
514348,sub-OAS30603_sess-d2253_CT__s5.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15


In [14]:
test_df

,NIFTI_slice_name,sid_x,NIFTI_name,sid_y,split,description,label
150,sub-OAS30303_ses-d0910_run-02_T1w__s11.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0
151,sub-OAS30303_ses-d0910_run-02_T1w__s22.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0
152,sub-OAS30303_ses-d0910_run-02_T1w__s23.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0
153,sub-OAS30303_ses-d0910_run-02_T1w__s14.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0
154,sub-OAS30303_ses-d0910_run-02_T1w__s21.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0
...,...,...,...,...,...,...,...
514095,sub-OAS30579_sess-d3440_run-01_FLAIR__s14.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3
514096,sub-OAS30579_sess-d3440_run-01_FLAIR__s0.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3
514097,sub-OAS30579_sess-d3440_run-01_FLAIR__s2.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3
514098,sub-OAS30579_sess-d3440_run-01_FLAIR__s6.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3


In [15]:
orig_train_df = pd.read_csv('Oasis3_trainLabels.txt',names=['ID','label','extra'],sep='\t',
                            dtype={'ID':str,'label':int,'extra':int})
orig_test_df = pd.read_csv('Oasis3_testLabels.txt', names=['ID', 'label', 'extra'], sep='\t',
                            dtype={'ID': str, 'label': int, 'extra': int})

In [16]:
orig_train_df['sid'] = orig_train_df['ID'].apply(lambda x: os.path.split(x)[1])
orig_train_df

,ID,label,extra,sid
0,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30333_ses-d141_acq-PIB_pet__s6.nii.gz
1,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30335_ses-d2783_T2star__s18.nii.gz
2,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30821_ses-d0068_T2star__s11.nii.gz
3,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30250_sess-d3004_echo-2_run-01_fieldmap...
4,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30445_ses-d0133_acq-AV45_pet__s14.nii.gz
...,...,...,...,...
459970,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30505_ses-d4682_run-03_T1w__s17.nii.gz
459971,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30705_sess-d2185_T2star__s22.nii.gz
459972,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30758_ses-d0007_T2w__s7.nii.gz
459973,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30704_ses-d0584_T2w__s6.nii.gz


In [17]:
orig_test_df['sid'] = orig_test_df['ID'].apply(lambda x: os.path.split(x)[1])
orig_test_df

,ID,label,extra,sid
0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30303_ses-d0910_run-02_T1w__s11.nii.gz
1,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS309...,0,0,sub-OAS30928_ses-d0177_T1w__s4.nii.gz
2,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30371_ses-d0338_run-01_T1w__s19.nii.gz
3,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0,sub-OAS30579_ses-d2400_T2star__s6.nii.gz
4,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30376_ses-d0082_echo-2_run-01_fieldmap_...
...,...,...,...,...
54370,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0,sub-OAS30579_sess-d3440_task-restingstateMB4_r...
54371,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS309...,0,0,sub-OAS30900_ses-d0051_T2star__s7.nii.gz
54372,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS302...,0,0,sub-OAS30251_ses-d0220_task-rest_run-03_bold__...
54373,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS302...,0,0,sub-OAS30222_ses-d1100_acq-PIB_pet__s12.nii.gz


In [18]:
train_df_merged = train_df.merge(orig_train_df, left_on='NIFTI_slice_name', right_on='sid', how='inner')
train_df_merged

,NIFTI_slice_name,sid_x,NIFTI_name,sid_y,split,description,label_x,ID,label_y,extra,sid
0,sub-OAS30333_ses-d141_acq-PIB_pet__s6.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30333_ses-d141_acq-PIB_pet__s6.nii.gz
1,sub-OAS30333_ses-d141_acq-PIB_pet__s9.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30333_ses-d141_acq-PIB_pet__s9.nii.gz
2,sub-OAS30333_ses-d141_acq-PIB_pet__s22.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30333_ses-d141_acq-PIB_pet__s22.nii.gz
3,sub-OAS30333_ses-d141_acq-PIB_pet__s4.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30333_ses-d141_acq-PIB_pet__s4.nii.gz
4,sub-OAS30333_ses-d141_acq-PIB_pet__s7.nii.gz,sub-OAS30333,sub-OAS30333_ses-d141_acq-PIB_pet.nii.gz,sub-OAS30333,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30333_ses-d141_acq-PIB_pet__s7.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...
459970,sub-OAS30603_sess-d2253_CT__s1.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30603_sess-d2253_CT__s1.nii.gz
459971,sub-OAS30603_sess-d2253_CT__s3.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30603_sess-d2253_CT__s3.nii.gz
459972,sub-OAS30603_sess-d2253_CT__s4.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30603_sess-d2253_CT__s4.nii.gz
459973,sub-OAS30603_sess-d2253_CT__s5.nii.gz,sub-OAS30603,sub-OAS30603_sess-d2253_CT.nii.gz,sub-OAS30603,train,NotMri,15,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0,sub-OAS30603_sess-d2253_CT__s5.nii.gz


In [19]:
test_df_merged = test_df.merge(orig_test_df, left_on='NIFTI_slice_name', right_on='sid', how='inner')
test_df_merged

,NIFTI_slice_name,sid_x,NIFTI_name,sid_y,split,description,label_x,ID,label_y,extra,sid
0,sub-OAS30303_ses-d0910_run-02_T1w__s11.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30303_ses-d0910_run-02_T1w__s11.nii.gz
1,sub-OAS30303_ses-d0910_run-02_T1w__s22.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30303_ses-d0910_run-02_T1w__s22.nii.gz
2,sub-OAS30303_ses-d0910_run-02_T1w__s23.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30303_ses-d0910_run-02_T1w__s23.nii.gz
3,sub-OAS30303_ses-d0910_run-02_T1w__s14.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30303_ses-d0910_run-02_T1w__s14.nii.gz
4,sub-OAS30303_ses-d0910_run-02_T1w__s21.nii.gz,sub-OAS30303,sub-OAS30303_ses-d0910_run-02_T1w.nii.gz,sub-OAS30303,test,T1w,0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS303...,0,0,sub-OAS30303_ses-d0910_run-02_T1w__s21.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...
54370,sub-OAS30579_sess-d3440_run-01_FLAIR__s14.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0,sub-OAS30579_sess-d3440_run-01_FLAIR__s14.nii.gz
54371,sub-OAS30579_sess-d3440_run-01_FLAIR__s0.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0,sub-OAS30579_sess-d3440_run-01_FLAIR__s0.nii.gz
54372,sub-OAS30579_sess-d3440_run-01_FLAIR__s2.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0,sub-OAS30579_sess-d3440_run-01_FLAIR__s2.nii.gz
54373,sub-OAS30579_sess-d3440_run-01_FLAIR__s6.nii.gz,sub-OAS30579,sub-OAS30579_sess-d3440_run-01_FLAIR.nii.gz,sub-OAS30579,test,T2-FLAIR,3,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS305...,0,0,sub-OAS30579_sess-d3440_run-01_FLAIR__s6.nii.gz


In [20]:
train_df_merged[['ID','label_x','extra']].to_csv('Oasis3_trainLabels_extensive3.txt',index=False, header=False, sep='\t')
test_df_merged[['ID','label_x','extra']].to_csv('Oasis3_testLabels_extensive3.txt',index=False, header=False, sep='\t')